<a href="https://colab.research.google.com/github/rushi-k12/API/blob/exploration_various_optimizations/API_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

<ipython-input-15-7ca7f5bd7fbb>:2: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df['timestamp'] = pd.to_datetime('now')


,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,...,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,10863,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",21000000.00,19647368.00,19647368.00,...,52.13,1410654538117.91,None,2024-03-07T14:23:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
1,1027,Ethereum,ETH,ethereum,8511,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,120114025.49,120114025.49,...,18.15,459818528052.36,None,2024-03-07T14:23:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
2,825,Tether USDt,USDT,tether,77422,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,100729057191.32,103797122782.55,...,3.98,103869999830.96,None,2024-03-07T14:22:00.000Z,1027.00,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2024-03-07 14:23:33.793385
3,1839,BNB,BNB,bnb,2085,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,149540856.10,149540856.10,...,2.65,67155412269.44,None,2024-03-07T14:22:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
4,5426,Solana,SOL,solana,635,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,443013429.57,571205232.37,...,2.57,83729856389.48,None,2024-03-07T14:23:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
5,52,XRP,XRP,xrp,1243,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",100000000000.00,54664031281.00,99987852435.00,...,1.35,62422540933.97,None,2024-03-07T14:23:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
6,3408,USDC,USDC,usd-coin,17081,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,29135978377.86,29135978377.86,...,1.15,29136583843.36,None,2024-03-07T14:23:00.000Z,1027.00,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2024-03-07 14:23:33.793385
7,2010,Cardano,ADA,cardano,1090,2017-10-01T00:00:00.000Z,"[dpos, pos, platform, research, smart-contract...",45000000000.00,35489228193.02,36685854736.85,...,1.03,33241467833.78,None,2024-03-07T14:22:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
8,74,Dogecoin,DOGE,dogecoin,914,2013-12-15T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,143369266383.71,143369266383.71,...,0.88,22229099309.81,None,2024-03-07T14:23:00.000Z,NaN,NaN,NaN,NaN,NaN,2024-03-07 14:23:33.793385
9,5994,Shiba Inu,SHIB,shiba-inu,756,2020-08-01T00:00:00.000Z,"[memes, ethereum-ecosystem, doggone-doggerel]",NaN,589289410812691.00,589534086491242.25,...,0.73,18606214277.88,None,2024-03-07T14:22:00.000Z,1027.00,Ethereum,ETH,ethereum,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,2024-03-07 14:23:33.793385


In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df.columns

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply',
       'infinite_supply', 'platform', 'cmc_rank',
       'self_reported_circulating_supply', 'self_reported_market_cap',
       'tvl_ratio', 'last_updated', 'quote.USD.price', 'quote.USD.volume_24h',
       'quote.USD.volume_change_24h', 'quote.USD.percent_change_1h',
       'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d',
       'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d',
       'quote.USD.percent_change_90d', 'quote.USD.market_cap',
       'quote.USD.market_cap_dominance', 'quote.USD.fully_diluted_market_cap',
       'quote.USD.tvl', 'quote.USD.last_updated', 'platform.id',
       'platform.name', 'platform.symbol', 'platform.slug',
       'platform.token_address', 'timestamp'],
      dtype='object')

In [ ]:


# Feature selection/engineering
# Select relevant features
# Drop the target variable from the features
features = df.drop(columns=['quote.USD.price'])
# Other feature engineering steps...

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, df['quote.USD.price'], test_size=0.2, random_state=42)

In [ ]:
# Define the columns to keep
selected_columns = [
    'id', 'num_market_pairs', 'circulating_supply',
    'total_supply', 'infinite_supply', 'cmc_rank',
    'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
    'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
    'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
    'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
    'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
    'quote.USD.fully_diluted_market_cap'
]

# Select only the desired columns
X_test_numeric = X_test.loc[:, selected_columns]
X_train_numeric = X_train.loc[:, selected_columns]

In [ ]:
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Define the base model
base_model = XGBRegressor()

# Define the bagging ensemble model
bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42)

# Train the ensemble model
bagging_model.fit(X_train, y_train)

# Make predictions
predictions = bagging_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ValueError: could not convert string to float: 'Chainlink'

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Define the base models for the first layer
base_models_layer1 = [
    ('xgb', XGBRegressor()),
    ('dt', DecisionTreeRegressor())
]

# Define the meta-model for the first layer
meta_model_layer1 = LinearRegression()

# Define the stacking ensemble model for the first layer
stacking_model_layer1 = StackingRegressor(estimators=base_models_layer1, final_estimator=meta_model_layer1)

# Define the bagging ensemble model for the second layer
bagging_model_layer2 = BaggingRegressor(base_estimator=stacking_model_layer1, n_estimators=10, random_state=42)

# Define the boosting ensemble model for the third layer
boosting_model_layer3 = AdaBoostRegressor(base_estimator=bagging_model_layer2, n_estimators=50, random_state=42)

# Train the ensemble model
boosting_model_layer3.fit(X_train, y_train)

# Make predictions
predictions = boosting_model_layer3.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


ValueError: Input contains NaN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define preprocessing steps
preprocessor = Pipeline([
    ('scaler', StandardScaler())  # Standardize features
])

# Define the model
model = RandomForestRegressor()

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Define the hyperparameter grid
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10, 20],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions
predictions = best_model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


ValueError: 
All the 540 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-packages/joblib/memory.py", line 353, in __call__
    return self.func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 89

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Define your features and target variable
X = data.drop(columns=['Quote-USD-price'])
y = data['Quote-USD-price']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps
preprocessor = StandardScaler()

# Define the model
model = RandomForestRegressor()

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
predictions = pipeline.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


AttributeError: 'dict' object has no attribute 'drop'

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Load your dataset and split into features (X) and target variable (y)
# Replace X and y with your dataset's features and target variable
# X, y = load_your_dataset()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('regressor', DecisionTreeRegressor())  # Decision Tree regressor
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


NameError: name 'X' is not defined

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Define the base models
base_models = [
    ('xgb', XGBRegressor()),
    ('rf', RandomForestRegressor())
]

# Define the meta-model
meta_model = LinearRegression()

# Define the stacking ensemble model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

# Train the stacking ensemble model
stacking_model.fit(X_train, y_train)

# Make predictions
predictions = stacking_model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:name: object, symbol: object, slug: object, date_added: object, tags: object, tvl_ratio: object, last_updated: object, quote.USD.tvl: object, quote.USD.last_updated: object, platform.name: object, platform.symbol: object, platform.slug: object, platform.token_address: object, timestamp: datetime64[ns]

In [ ]:
model_3 = pipeline.fit(X_train_numeric, y_train)
model_3

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('scaler', StandardScaler())])),
                ('model', RandomForestRegressor())])

In [ ]:
model_2 = stacking_model.fit(X_train_numeric, y_train)
model_2

StackingRegressor(estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                              ('rf', RandomForestRegressor())],
                  final_estimator=LinearRegression())

In [ ]:
model_1 = base_model.fit(X_train_numeric, y_train)
model_1

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
model_3 = base_model.fit(X_train_numeric, y_train)
model_3

In [ ]:
model = xgb_regressor.fit(X_train_numeric, y_train)
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [ ]:
import joblib

# Define the file path where you want to save the model
model_file_path = 'xgb_regressor_model.pkl'

# Save the trained model to the specified file path
joblib.dump(xgb_regressor, model_file_path)


['xgb_regressor_model.pkl']

In [ ]:
import joblib

# Define the file path where you want to save the model
model_file_path = 'xgb_regressor_model_3.pkl'

# Save the trained model to the specified file path
joblib.dump(pipeline, model_file_path)

['xgb_regressor_model_3.pkl']

In [ ]:
import joblib

# Define the file path where you want to save the model
model_file_path = 'xgb_regressor_model_1.pkl'

# Save the trained model to the specified file path
joblib.dump(base_model, model_file_path)


['xgb_regressor_model_1.pkl']

In [ ]:
import joblib

# Define the file path where you want to save the model
model_file_path = 'xgb_regressor_model_2.pkl'

# Save the trained model to the specified file path
joblib.dump(stacking_model, model_file_path)

['xgb_regressor_model_2.pkl']

In [ ]:
# Load the saved model from the file path
loaded_model = joblib.load(model_file_path)

# Make predictions using the loaded model
predictions = loaded_model.predict(new_data_processed)

# Print or use the predictions
print("Predictions:", predictions)


NameError: name 'new_data_processed' is not defined

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return

    # Create DataFrame from fetched data
    df2 = pd.json_normalize(data['data'])
    df2['Timestamp'] = pd.to_datetime('now')

    # Append new data to the global DataFrame
    df = df.append(df2)

    # Check if the number of iterations has reached a threshold
    if len(df) % 100 == 0:
        # Define the file path
        csv_file_path = '/content/api_data.csv'

        # Check if the CSV file exists
        if not os.path.isfile(csv_file_path):
            # If it doesn't exist, create the file with headers
            df.to_csv(csv_file_path, header='column_names', index=False)
        else:
            # If it exists, append data to the file without writing headers
            df.to_csv(csv_file_path, mode='a', header=False, index=False)
            # Clear the DataFrame after saving to avoid duplicating data
            df = pd.DataFrame()

# Main loop to fetch data
for i in range(333):
    api_runner()


<ipython-input-39-037925cb3345>:26: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df2['Timestamp'] = pd.to_datetime('now')
<ipython-input-39-037925cb3345>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
<ipython-input-39-037925cb3345>:26: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  df2['Timestamp'] = pd.to_datetime('now')
<ipython-input-39-037925cb3345>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df2)
<ipython-input-39-037925cb3345>:26: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future 

KeyError: 'data'

In [ ]:
import joblib

# Load the trained model
model = joblib.load('xgb_regressor_model.pkl')

# Fetch new data from the API
new_data = api_runner()

# Preprocess the new data (assuming preprocess_new_data() is defined)
preprocessed_data = preprocess_new_data(new_data)

# Make predictions using the loaded model
predictions = model.predict(preprocessed_data)

# Optionally, perform post-processing on the predictions
# For example, convert predictions to a human-readable format or save them to a database

# Print or return the predictions
print("Predictions:", predictions)


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None


    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model_3.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


NameError: name 'preprocess_new_data' is not defined

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None


    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model_3.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


NameError: name 'preprocess_new_data' is not defined

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

# Function to fetch new data from the API
def fetch_new_data_from_api():
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        return data['data']
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        return None


    # Select relevant features
    selected_columns = [
        'id', 'num_market_pairs', 'circulating_supply',
        'total_supply', 'infinite_supply', 'cmc_rank',
        'quote.USD.volume_24h', 'quote.USD.volume_change_24h',
        'quote.USD.percent_change_1h', 'quote.USD.percent_change_24h',
        'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d',
        'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d',
        'quote.USD.market_cap', 'quote.USD.market_cap_dominance',
        'quote.USD.fully_diluted_market_cap'
    ]
    X_test_numeric = df.loc[:, selected_columns]

    return X_test_numeric

# Function to make predictions using the loaded model
def make_predictions(model, new_data):
    preprocessed_data = preprocess_new_data(new_data)
    predictions = model.predict(preprocessed_data)
    return predictions

# Load the trained model from the .pkl file
model_file_path = 'xgb_regressor_model_3.pkl'
loaded_model = joblib.load(model_file_path)

# Fetch new data from the API
new_data = fetch_new_data_from_api()

# Make predictions using the loaded model
if new_data is not None:
    predictions = make_predictions(loaded_model, new_data)
    print("Predictions:", predictions)
else:
    print("Failed to fetch new data from the API.")


NameError: name 'preprocess_new_data' is not defined